In [42]:
import plotly 
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.io as pio
import os

# for ploting online plots, account info is needed
# plotly.tools.set_credentials_file(username='ariellilili', api_key='X8MtW4k2mpBL5tUUHZdi')

# create offline plots
plotly.offline.init_notebook_mode(connected=True)

# import data
face = pd.read_csv('face_camera.csv')

# change timestamp datatype to datetime
face['timestamp'] = pd.to_datetime(face['timestamp'])
face.head()

,timestamp,cameraID,faceID,gender,age,customerType,mood,accompany,zoneID
0,2019-05-10 10:00:10,1,L7O49N,Male,50,新客,中性,无伴,outdoor
1,2019-05-10 10:00:38,1,FGROAE,Female,30,新客,中性,无伴,outdoor
2,2019-05-10 10:02:16,1,POSH4U,Female,44,新客,中性,有伴,outdoor
3,2019-05-10 10:02:33,1,NFS57J,Unknown,53,老客,开心,无伴,outdoor
4,2019-05-10 10:03:13,1,VTG62R,Male,29,新客,中性,无伴,outdoor


In [49]:
# indoor rate
df = face
in_rate = "{:.0%}".format(df[df['zoneID']=='in'].shape[0]/df[df['zoneID']=='outdoor'].shape[0])
# calculate per hour visits
df.index = df['timestamp']
df = df['faceID'].groupby(df['zoneID']).resample('H').count().to_frame()

data = [go.Scatter(
    x=df.loc['outdoor'].index,
    y=df.loc['outdoor']['faceID'],
    mode = 'lines+markers',
    name = '门口人流量',
    marker=dict(color='#454552')
    ),
        go.Scatter(
    x=df.loc['in'].index,
    y=df.loc['in']['faceID'],
    mode = 'lines+markers',
            name = '进门数',
            marker=dict(color='#e85a71')
    )
       ]
layout = go.Layout(
    title='每小时访客，总进门率：'+in_rate,
    font=dict(family='微软雅黑')
)

fig1 = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig1)
pio.write_image(fig1, '单日访客.jpeg', scale=5, width=600, height = 400)

In [44]:
# plot age/gender pyramid

# get data
df = face[face['zoneID'] == 'in']
women_bins, bins = np.histogram(df[df['gender'] == 'Female']['age'], bins=list(range(10, 80, 10)))
men_bins, bins = np.histogram(df[df['gender'] == 'Male']['age'], bins=list(range(10, 80, 10)))
women_bins = -women_bins
y = bins

# plot
layout = go.Layout(title="年龄与性别",
                   yaxis=go.layout.YAxis(title='年龄'),
                   xaxis=go.layout.XAxis(
                       showticklabels=False,
                       showgrid=False,
                       title='访客数'),
                   barmode='overlay',
                   bargap=0.1,
                  font=dict(family='微软雅黑'))

data = [go.Bar(y=y,
               x=men_bins,
               orientation='h',
               name='男',
               hoverinfo='x',
               text=men_bins,
               textposition = 'auto',
               marker=dict(color='#4ea1d3'),
               opacity=0.8
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='女',
               text=-1 * women_bins.astype('int'),
               hoverinfo='text',
               textposition = 'auto',
               marker=dict(color='#e85a71'),
               opacity=0.8
               )]

fig2 = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig2)
pio.write_image(fig2, '年龄性别.jpeg', scale=5, width=600, height = 400)

In [45]:
# donut plots for customer type, mood, and accompany

# prepare data
df = face[face['zoneID'] == 'in']
df1 = df['customerType'].value_counts().to_frame()
df2 = df['mood'].value_counts().to_frame()
df3 = df['accompany'].value_counts().to_frame()

fig3 = {
  "data": [
    {
      "values": df1['customerType'],
      "labels": df1.index,
      "domain": {"column": 0},
      "name": "Customer Type",
        'textinfo' : 'label+percent',
        'showlegend' : False,
      "hoverinfo":"value",
      "hole": .5,
      "type": "pie",
        'marker': {'colors': ['#d8e9ef',
                                  '#4ea1d3']}
    },
    {
      "values": df2['mood'],
      "labels": df2.index,
        'textinfo' : 'label+percent',
        'showlegend' : False,
      "hoverinfo":"value",
      "domain": {"column": 1},
      "name": "Mood",
      "hole": .5,
      "type": "pie",
        'marker': {'colors': ['#d8e9ef',
                                  '#4ea1d3']}
                  },
        {
      "values": df3['accompany'],
      "labels": df3.index,
        'textinfo' : 'label+percent',
        'showlegend' : False,
      "hoverinfo":"value",
      "domain": {"column": 2},
      "name": "Mood",
      "hole": .5,
      "type": "pie",
        'marker': {'colors': ['#d8e9ef',
                                  '#4ea1d3']}
        }
    ],

  "layout": {
        "grid": {"rows": 1, "columns": 3},
      "font": {'family': '微软雅黑'},
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "回头客",
                "x": 0.12,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "情绪",
                "x": 0.5,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "有伴与否",
                "x": 0.9,
                "y": 0.5
            }
        ]
    }
}
plotly.offline.iplot(fig3)
pio.write_image(fig3, '3pie.jpeg', scale=5, width=1000, height = 400)